In [ ]:
import os
import pickle
import re
import numpy as np
try:
    from rouge import Rouge
except:
    !pip install rouge
    from rouge import Rouge

In [ ]:
VECTOR_PATH = '../glove.6B.200d.pkl'
ARTICLES_PATH = '../articles/'
GOLD_SUMMARIES_PATH = '../gold_summaries/'

In [ ]:
def read_documents(doc_folder):
    """Read documents
    
    Read documents in the given `doc_folder`
    
    Parameters
    ----------
    doc_folder: str
        Document folder path
        
    Returns
    -------
    list
        Documents
    """
    article_names = os.listdir(doc_folder)
    article_names.sort()
    
    documents = []

    for article_name in article_names:
        path = os.path.join(doc_folder, article_name)
        with open(path, "rb") as f:
            try:
                doc = f.read().decode("utf-8")
            except UnicodeDecodeError as e:
                print(article_name, " has unknown encoding")
                raise e
        documents.append(doc)

    return documents

In [ ]:
def parse_document_to_sentence(doc):
    """Parse document to sentences
    
    This parser assumes every sentence begins with upper-case or digit and
    ends with ., ..., ?, ! or whitespace and after each sentence there 
    must be a whitespace.
    
    Parameters
    ----------
    doc: str
        Document string to parse
        
    Returns
    -------
    list
        Sentences
    """
    pat = re.compile(r"([A-Z0-9].*?)[\.?!\s]+\s", re.MULTILINE | re.DOTALL)
    
    return pat.findall(doc)

In [ ]:
def parse_documents_to_sentences(docs_path):
    """Parse documents

    Extract sentences for given documents folders path

    Parameters
    ----------
    docs_path: str
        Path of folder that contains documents

    Returns
    -------
    list
        Documents parsed to their sentences as numpy array
    """
    docs = read_documents(docs_path)
    return [np.array(parse_document_to_sentence(doc)) for doc in docs]

In [ ]:
def sent2vec(sent, word2vec):
    """Sentence to vector
    
    Transform plain sentence to its vector representation
    
    Parameters
    ----------
    sent: str
        Sentence
    word2vec: dict
        Word vectors

    Returns
    -------
    numpy.ndarray
        Vector representation of sentence
    """
    tokens = sent.lower().split()
    
    return np.mean(
        [
            word2vec[token] if token in word2vec
            else word2vec["unk"]
            for token in tokens
        ],
        axis=0
    )

In [ ]:
def doc2vec(docs_by_sentences, word2vec):
    """Documents to vectors

    Convert sentences of documents to vectors

    Parameters
    ----------
    docs_by_sentences: list
        List of list of sentences

    word2vec: dict
        Word vectors

    Returns
    -------
    list
        List of numpy.array, each numpy.array is document representation by its sentences
    """
    docs_as_vectors = []

    return [np.array([sent2vec(sent, word2vec) for sent in doc]) for doc in docs_by_sentences]

In [ ]:
with open(VECTOR_PATH, "rb") as f:
    word_vectors = pickle.load(f)

docs = read_documents(ARTICLES_PATH)

sentences = parse_documents_to_sentences(ARTICLES_PATH)

doc_vectors = doc2vec(sentences, word_vectors)

# K-Means Clustering
## Implementation

The loss function is:

$\mathcal{L}$ = ${\left|\left| X - CM \right|\right|}_{F}$

$N$ is the number of sample, $D$ is the input dimension, $k$ is the number of clusters

$C \in \{0, 1\}^{N \times k}$, $M \in \mathbb{R}^{k \times D}$

The only constraint we have is $\sum_{i}{C_{n, i} = 1}$

\begin{align*}
    \mathcal{L}^2 &= {\left|\left| X - CM \right|\right|}_{F}^2 \\
                &= trace((X - CM)^T (X - CM)) \\
                &= trace(X^T X - X^T CM - M^T C^T X + M^T C^T C M)
\end{align*}

The partial derivative w.r.t $M$ follows

\begin{align*}
    \frac{\partial \mathcal{L}^2}{\partial M} &= \frac{\partial trace(X^T X - X^T CM - M^T C^T X + M^T C^T C M)}{\partial M} \\
                                            &= \frac{\partial trace(X^T X)}{\partial M} + \frac{\partial trace(- X^T CM)}{\partial M} + \frac{\partial trace(- M^T C^T X)}{\partial M} + \frac{\partial trace(M^T C^T C M)}{\partial M} \\
                                            &= 0 - C^T X - C^T X + 2 C^T C M \\
                                            &= -2 C^T X + 2 C^T C M
\end{align*}

And let derivative equals 0


\begin{align*}
    \frac{\partial \mathcal{L}^2}{\partial M} &= -2 C^T X + 2 C^T C M = 0 \\
    &\Rightarrow C^T C M = C^T X \\
    &\Rightarrow M = (C^T C)^{-1} C^T X
\end{align*}

Clusters have been assigned w.r.t. distance by calculated cluster means, which are rows of matrix $M$. So, for each example number $n$, corresponding value in assignment matrix is calculated as $C_{n, k} = 1$ and $C_{n, i} = 0$ for $k \neq i$, where $k = \arg\min_{k}{\left|\left| X_{n, :} - M_{k, :} \right|\right|}$. So, we can find cluster assignment in vectorized way as follows:

\begin{align*}
    \arg\min_{k}{{\left|\left| X_{n} - M_{k} \right|\right|}_2} &=  \arg\min_{k}{{\left|\left| X_{n} - M_{k} \right|\right|}_2^2}\\
                                                   &= \arg\min_{k}{\sum_{i}{(X_{n, i} - M_{k, i})^2}} \\
                                                   &= \arg\min_{k}{\sum_{i}{X_{n, i}^2 - 2 X_{n, i} M_{k, i} + M_{k, i}^2}} \\
                                                   &= \arg\min_{k}{\sum_{i}{- 2 X_{n, i} M_{k, i} + M_{k, i}^2}} \\
                                                   &= \arg\min_{k}{\sum_{i}{- 2 X_{n, i} M^T_{i, k} + M_{k, i}^2}} \\
                                                   &= \arg\min_{k}{- 2 {X M^T}_{n, k} + \sum_{i}{M_{k, i}^2}}
\end{align*}

In [ ]:
k = 10
num_samples = 100
np.random.multinomial(1, pvals=np.ones(k) / k, size=num_samples).shape

In [ ]:
def to_onehot(vec, k):
    """Convert to onehot
    
    Converts `vec` to its onehot representation
    
    Parameters
    ----------
    vec: numpy.ndarray
    
    k: int
        Size of onehot vectors
    Returns
    -------
    numpy.ndarray
        Onehot representation of given array
    """
    one_hot_mtx = np.zeros((vec.shape[0], k))
    for sample, idx in enumerate(vec):
        one_hot_mtx[sample, idx] = 1
        
    return one_hot_mtx

In [ ]:
def K_means(data, k, verbose=False):
    """K-Means clustering
    
    Given the `data` in shape (N, D) where N is the number of examples
    and D is the number of features, it calculates `k` cluster center
    that represents the data most. Maths behind this implementation is detailed
    above.
    
    Parameters
    ----------
    data: numpy.ndarray
        Data
    k: int
        Number of Clusters
    Verbose: bool
        Print loss in each iteration if True
        
    Returns
    -------
    M: numpy.ndarray
        Cluster centers
    C: numpy.ndarray
        Data assignments
    loss: float
        Last calculated loss
    """
    num_samples, input_dim = data.shape
    

    M = np.random.rand(k, input_dim)
    C = to_onehot(np.argmin(-2 * np.dot(data, M.T) + np.power(M, 2).sum(axis=1)[np.newaxis, :], axis=1), k)
    prev_C = np.zeros_like(C)
    
    loss = np.linalg.norm(data - np.dot(C, M), ord="fro")
    
    while not np.all(C == prev_C):
        if verbose:
            print("Loss: {}".format(loss))
        prev_C = C
        
        M = np.dot(np.dot(np.linalg.pinv(np.dot(C.T, C)), C.T), data)
        C = to_onehot(np.argmin(-2 * np.dot(data, M.T) + np.power(M, 2).sum(axis=1)[np.newaxis, :], axis=1), k)

        loss = np.linalg.norm(data - np.dot(C, M), ord="fro")
        
    return M, C, loss

## Testing K-Means

In this part, I have created fake dataset and tested my implementation of k-means clustering algorithm with the fake dataset

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# 3 Different centers
data1 = np.random.rand(50, 2) * 4 + 5
data2 = np.random.rand(30, 2) * 4
data3 = np.random.rand(40, 2) * 4 + 10

data = np.concatenate([data1, data2, data3])

### Plot dataset

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(data[:, 0], data[:, 1], "o")

### Plot dataset with cluster centers

In [ ]:
M, C, loss = K_means(data, 3)
plt.figure(figsize=(20, 10))
plt.plot(data[:, 0], data[:, 1], "o")
plt.plot(M[:, 0], M[:, 1], "rx")

In [ ]:
def find_most_repr_sent_in_cluster(data, M):
    """Find Most Representative Sentences
    
    Parameters
    ----------
    data: numpy.ndarray
        Document representation
        
    M: numpy.ndarray
        Cluster centers
        
    Returns
    -------
    numpy.ndarray
        Indices of sentences which are most representative ones
    """
    return np.argmin(-2 * np.dot(data, M.T) + np.power(M, 2).sum(axis=1)[np.newaxis, :], axis=0)

In [ ]:
def cluster(doc_vectors, num_k_means_trial=5):
    """Cluster and return most represntative sentence indices
    
    Parameters
    ----------
    doc_vectors: numpy.ndarray
        Document representation by sentence vectors
        
    num_k_means_trial: int
        Number of k-means trial select best one
        
    Returns
    -------
    numpy.ndarray
        List of most representative indices
    """
    k = max(1, int(len(doc_vectors) ** 0.5))

    k_means = []
    min_loss_model = -1
    min_loss = float("inf")

    for i in range(num_k_means_trial):
        M, C, loss = K_means(doc_vectors, k)
        if loss < min_loss:
            min_loss = loss
            min_loss_model = i
            
        k_means.append(M)
        
    best_means = k_means[min_loss_model]
    repr_sentences = find_most_repr_sent_in_cluster(doc_vectors, best_means)

    return repr_sentences

In [ ]:
def extract_summary(doc_vectors, doc_sentences):
    """Extracts Summary
    
    Parameters
    ----------
    doc_vectors: numpy.ndarray
        Document representation by sentence vectors
    doc_sentences: numpy.ndarray
        Document representation by its sentences
        
    Returns
    -------
    str
        Summary of document
    """
    most_repr_sentences = cluster(doc_vectors)
    return ". ".join(list(set(doc_sentences[most_repr_sentences])))

In [ ]:
summaries = [extract_summary(doc_vector, doc_sentences) for doc_vector, doc_sentences in zip(doc_vectors, sentences)]
gold_summaries = read_documents(GOLD_SUMMARIES_PATH)

In [ ]:
def evaluation(gold_summaries, extracted_summaries):
    """Evaluation of summaries
    
    Parameters
    ----------
    gold_summaries: list
        Reference summaries
    extracted_summaries: list
        Extracted summaries by the model
        
    Returns
    -------
    list
        Rough scores as tuple (rouge-1.f, rouge-2.f, rouge-l.f)
    """
    rouge = Rouge()
    rouge_scores = []
    for hyp, ref in zip(gold_summaries, extracted_summaries):
        scores = rouge.get_scores(hyp, ref)[0]
        rouge_scores.append((scores["rouge-1"]["f"], scores["rouge-2"]["f"], scores["rouge-l"]["f"]))
    return rouge_scores

In [ ]:
r_scores = evaluation(gold_summaries, summaries)
r_scores_avg = np.array(r_scores).mean(axis=0)
r_scores_avg

In [ ]:
# Construct at least two different models and compare their results. End with a conclusion. You are
# encouraged to use plots/histograms etc. for coparison or evaluation.